## This code creates a datetime column on the Origin and Destination data (from OnTime)
The Origin data is a copy of the OnTime data containing only the Origin, Flight Date, and CRSDepTime (departure time).<br>
The Destination data is also a copy of the OnTime data containing three fields, Destination, Flight Date, and CRSArrTime (arrival time).<br>
* It would have been better to give this file the departure time too. These columns are for linking back to the OnTime data as well as calculating the time the weather report is wanted.

In [ ]:
import pandas as pd
basepath = 'your_path_here'

## Create a function to run the same process for origin and then destination for each year

# The date and time provided in the OnTime data are strings and not formatted as date types
This code takes the two strings and creates a datetime column.

If a date can't be created from the date and time provided then set it to July 20, 2023 at 1:01am. This will not match up to any weather data.
Weather data goes up till 6/17, by making the date June 20th for those that a date doesn't work for, it will not match to weather data and not return erroneous weather, only missing data.

In [ ]:
weather_cols = ['w_dir_angle', 'w_type', 'w_speed_rate', 'sky_c_hgt', 'sky_c_det', 'sky_c_cavok', 'vis_dist', 'vis_var', 'tmp_air', 'tmp_dew', 'sea_lvl_p', 'liq_precip_qty', 'liq_precip_dim', 'liq_precip_cond', 'sky_cov', 'sky_cov_base_hgt', 'sky_cov_cld', 'sky_sum_cov', 'sky_sum_hgt', 'sky_obs_tot_cov', 'sky_low_cld_base_hgt', 'at_pres_altimeter_rate', 'at_pres_stn_rate']
miss_vals =    [999,           '9',      9999,           99999,       '9',         '9',            999999,    '9',        9999,      9999,      99999,       '99',             9999,             9,                '99' ,      99999,                 '99',         9,             99999,         '99',               99999,                99999,                      99999]

In [ ]:
def createPredictTime(od_time, hours = 0, minutes = 0):
    try:
        PredictTime = od_time - pd.Timedelta(hours= hours, minutes= minutes)
    except:
        print("od_time:", od_time)
        PredictTime = pd.to_datetime("202307200101", format= '%Y%m%d%H%M')
    return PredictTime

# We need DtTm_cols to be a global list so it's being created outside the function
DtTm_cols = []
def createFlightWeatherData(weather_df, origdest, origdestcol, year, hrs, mins):
    # print(origdestcol, "Shape:",origdest.shape)
    # print("Time:", datetime.now())

    origin_yr = origdest[origdest['OriginDtTm'].dt.strftime('%Y') == year].copy()
    origin_yr.insert(len(origin_yr.columns), 'PredictTime', None)
    origin_yr['PredictTime'] = origin_yr['OriginDtTm'].apply(lambda x: createPredictTime(x, hours = hrs,  minutes = mins)).copy()
    # print("\t",origdestcol, "filtered to year shape:",origin_yr.shape)

    # Get list of airport codes from OnTime and list from Weather
    origin_airports = origin_yr[origdestcol].unique().tolist()
    weather_airports = weather_df['airport_code'].unique().tolist()

    # Create list of those that are not in weather - we'll want this list to know which airports to drop from the OnTime data for weather delay predicions.
    no_weather_airports = []
    for airport in origin_airports:
        if airport not in weather_airports:
            no_weather_airports.append(airport)
    # print("\tNo weather data for these airports:\n\t", no_weather_airports)

    airport_in_Weather_and_Flight = []
    for airport in origin_airports:
        if airport in weather_airports:
            airport_in_Weather_and_Flight.append(airport)
    origin_yr_weather_airports = origin_yr[(origin_yr[origdestcol].isin(airport_in_Weather_and_Flight) )]

    # all_airport_weather_df = pd.DataFrame()
    airport_df = pd.DataFrame()
    airport_cnt = 0
    # Even though this is set as a global list we only need the list populated from one run through.
    # We don't want the columns added to the list duplicating them.
    DtTm_cols = []
    for airport in airport_in_Weather_and_Flight:
        airport_cnt += 1
        # print("Airport Code #"+str(airport_cnt) +":", airport)
        airport_flight_df = origin_yr_weather_airports[origin_yr_weather_airports[origdestcol] == airport].copy()
        airport_flight_df.sort_values('PredictTime', inplace=True)
        airport_weather_df = weather_df[weather_df['airport_code'] == airport].copy()
        airport_weather_df.sort_values('WeatherDtTm', inplace=True)

        # converting this to the index so we can preserve the date_start_time columns so we can validate the merging logic
        airport_flight_df.index = airport_flight_df['PredictTime']
        airport_weather_df.index = airport_weather_df['WeatherDtTm']

        # check the direction and tolerance arguments - these are what get the 'closeness' or closest match
        # we're taking the row with the closest match to the time - equal to or prior, not a future datetime
        # the tolerance is used to indicate the maximum amount prior
        tol = pd.Timedelta('1 hour')

        for i in range(len(weather_cols)):
            weather_col_df = airport_weather_df[~(airport_weather_df[weather_cols[i]] == miss_vals[i])][['WeatherDtTm', weather_cols[i]]]
            weather_col_df.rename(columns={'WeatherDtTm': weather_cols[i]+'_DtTm' }, inplace=True)
            weather_col_df.index = weather_col_df[weather_cols[i]+'_DtTm']
            DtTm_cols.append(weather_cols[i]+'_DtTm')
            # print("airport_flight_df", airport_flight_df.shape)
            # print("weather_col_df", weather_col_df.shape)
            airport_flight_df = pd.merge_asof(left=airport_flight_df,right=weather_col_df,right_index=True,left_index=True,direction='backward',tolerance=tol,\
                suffixes=('_x_'+weather_cols[i], '_y_'+weather_cols[i] )) #,tolerance=tol)

            if isinstance(miss_vals[i], int):
                airport_flight_df[[weather_cols[i]]] = airport_flight_df[[weather_cols[i]]].fillna(miss_vals[i]).astype(int)
            else:
                airport_flight_df[[weather_cols[i]]] = airport_flight_df[[weather_cols[i]]].fillna(miss_vals[i]).astype(str)

        airport_df = pd.concat([airport_df, airport_flight_df], axis = 0)

    airport_df.reset_index(inplace=True, drop=True)
    if origdestcol == 'Origin':
        airport_df.to_pickle(basepath + "/Weather/Hourly/FlightWeatherOrig_"+ year+"_DFW_uniq_bf.pkl")
    else:
        airport_df.to_pickle(basepath + "/Weather/Hourly/FlightWeatherDest_"+ year+"_DFW_uniq_bf.pkl")
    print("\tFINAL SHAPE:",airport_df.shape)

    return airport_df, DtTm_cols


In [ ]:
# Bring in the OnTime data - one dataset for the origin and one for dest
#    Each contain the airport_code, FlightDate and the CRSDepTime or CRSArriveTime
#    Rename the datetime column created on each to match the ones on the weather files
origindatetime = pd.read_parquet(basepath + "/OnTime/origindatetime_dttm_dfw_uniq.parquet")
destdatetime = pd.read_parquet(basepath + "/OnTime/destdatetime_dttm_dfw_uniq.parquet")
origindatetime.rename({'OriginDtTm2': 'OriginDtTm'}, axis = 1, inplace= True)
destdatetime.rename({'OrigDtTm2': 'OriginDtTm'}, axis = 1, inplace= True)

## Now to run the code using the above functions

Run by individual year.

In [ ]:
weather_2010 = pd.read_parquet(basepath + "/Weather/Hourly/Weather_DFW_2010.parquet")
orig_2010_dfs, DtTm_cols = createFlightWeatherData(weather_2010, origindatetime, 'Origin', '2010', 0, 45)
dest_2010_dfs, DtTm_cols = createFlightWeatherData(weather_2010, destdatetime, 'Dest', '2010', 0, 45)

	FINAL SHAPE: (49473, 51)
	FINAL SHAPE: (112609, 51)


In [ ]:
weather_2011 = pd.read_parquet(basepath + "/Weather/Hourly/Weather_DFW_2011.parquet")
orig_2011_dfs, DtTm_cols = createFlightWeatherData(weather_2011, origindatetime, 'Origin', '2011', 0, 45)
dest_2011_dfs, DtTm_cols = createFlightWeatherData(weather_2011, destdatetime, 'Dest', '2011', 0, 45)

	FINAL SHAPE: (66178, 51)
	FINAL SHAPE: (158045, 51)


In [ ]:
weather_2012 = pd.read_parquet(basepath + "/Weather/Hourly/Weather_DFW_2012.parquet")
orig_2012_dfs, DtTm_cols = createFlightWeatherData(weather_2012, origindatetime, 'Origin', '2012', 0, 45)
dest_2012_dfs, DtTm_cols = createFlightWeatherData(weather_2012, destdatetime, 'Dest', '2012', 0, 45)

	FINAL SHAPE: (49328, 51)
	FINAL SHAPE: (110926, 51)


In [ ]:
weather_2013 = pd.read_parquet(basepath + "/Weather/Hourly/Weather_DFW_2013.parquet")
orig_2013_dfs, DtTm_cols = createFlightWeatherData(weather_2013, origindatetime, 'Origin', '2013', 0, 45)
dest_2013_dfs, DtTm_cols = createFlightWeatherData(weather_2013, destdatetime, 'Dest', '2013', 0, 45)

	FINAL SHAPE: (44825, 51)
	FINAL SHAPE: (94455, 51)


In [ ]:
weather_2014 = pd.read_parquet(basepath + "/Weather/Hourly/Weather_DFW_2014.parquet")
orig_2014_dfs, DtTm_cols = createFlightWeatherData(weather_2014, origindatetime, 'Origin', '2014', 0, 45)
dest_2014_dfs, DtTm_cols = createFlightWeatherData(weather_2014, destdatetime, 'Dest', '2014', 0, 45)

	FINAL SHAPE: (57758, 51)
	FINAL SHAPE: (135431, 51)


In [ ]:
weather_2015 = pd.read_parquet(basepath + "/Weather/Hourly/Weather_DFW_2015.parquet")
orig_2015_dfs, DtTm_cols = createFlightWeatherData(weather_2015, origindatetime, 'Origin', '2015', 0, 45)
dest_2015_dfs, DtTm_cols = createFlightWeatherData(weather_2015, destdatetime, 'Dest', '2015', 0, 45)

	FINAL SHAPE: (36542, 51)
	FINAL SHAPE: (76817, 51)


In [ ]:
weather_2016 = pd.read_parquet(basepath + "/Weather/Hourly/Weather_DFW_2016.parquet")
orig_2016_dfs, DtTm_cols = createFlightWeatherData(weather_2016, origindatetime, 'Origin', '2016', 0, 45)
dest_2016_dfs, DtTm_cols = createFlightWeatherData(weather_2016, destdatetime, 'Dest', '2016', 0, 45)

	FINAL SHAPE: (47018, 51)
	FINAL SHAPE: (116696, 51)


In [ ]:
weather_2017 = pd.read_parquet(basepath + "/Weather/Hourly/Weather_DFW_2017.parquet")
orig_2017_dfs, DtTm_cols = createFlightWeatherData(weather_2017, origindatetime, 'Origin', '2017', 0, 45)
dest_2017_dfs, DtTm_cols = createFlightWeatherData(weather_2017, destdatetime, 'Dest', '2017', 0, 45)

	FINAL SHAPE: (21791, 51)
	FINAL SHAPE: (34627, 51)


In [ ]:
weather_2018 = pd.read_parquet(basepath + "/Weather/Hourly/Weather_DFW_2018.parquet")
orig_2018_dfs, DtTm_cols = createFlightWeatherData(weather_2018, origindatetime, 'Origin', '2018', 0, 45)
dest_2018_dfs, DtTm_cols = createFlightWeatherData(weather_2018, destdatetime, 'Dest', '2018', 0, 45)

	FINAL SHAPE: (37269, 51)
	FINAL SHAPE: (58008, 51)


In [ ]:
weather_2019 = pd.read_parquet(basepath + "/Weather/Hourly/Weather_DFW_2019.parquet")
orig_2019_dfs, DtTm_cols = createFlightWeatherData(weather_2019, origindatetime, 'Origin', '2019', 0, 45)
dest_2019_dfs, DtTm_cols = createFlightWeatherData(weather_2019, destdatetime, 'Dest', '2019', 0, 45)

	FINAL SHAPE: (36288, 51)
	FINAL SHAPE: (52848, 51)


In [ ]:
weather_2023 = pd.read_parquet(basepath + "/Weather/Hourly/Weather_DFW_2023.parquet")
orig_2023_dfs, DtTm_cols = createFlightWeatherData(weather_2023, origindatetime, 'Origin', '2023', 0, 45)
dest_2023_dfs, DtTm_cols = createFlightWeatherData(weather_2023, destdatetime, 'Dest', '2023', 0, 45)

	FINAL SHAPE: (50202, 51)
	FINAL SHAPE: (85493, 51)


# JOIN
# DROP ADDITIONAL DTTM COLUMNS!
# Out To Parquet

In [ ]:
OnTime_Weather_DFW_ORIG_10_19 = pd.concat([orig_2010_dfs, orig_2011_dfs, orig_2012_dfs, orig_2013_dfs, orig_2014_dfs, orig_2015_dfs, orig_2016_dfs, orig_2017_dfs, orig_2018_dfs, orig_2019_dfs], axis=0)
OnTime_Weather_DFW_ORIG_10_19.drop(columns=DtTm_cols, inplace=True)
OnTime_Weather_DFW_ORIG_10_19.to_parquet(basepath + "/OnTime/OnTime_Weather_DFW_ORIG_10_19_bf.parquet")

orig_2023_dfs.drop(columns=DtTm_cols, inplace=True)
orig_2023_dfs.to_parquet(basepath + "/OnTime/OnTime_Weather_DFW_ORIG_23_bf.parquet")
OnTime_Weather_DFW_ORIG_All_Yrs = pd.concat([OnTime_Weather_DFW_ORIG_10_19, orig_2023_dfs], axis=0)
OnTime_Weather_DFW_ORIG_All_Yrs.to_parquet(basepath + "/OnTime/OnTime_Weather_DFW_ORIG_All_Yrs_bf.parquet")

In [ ]:
OnTime_Weather_DFW_DEST_10_19 = pd.concat([dest_2010_dfs, dest_2011_dfs, dest_2012_dfs, dest_2013_dfs, dest_2014_dfs, dest_2015_dfs, dest_2016_dfs, dest_2017_dfs, dest_2018_dfs, dest_2019_dfs], axis=0)
OnTime_Weather_DFW_DEST_10_19.drop(columns=DtTm_cols, inplace=True)
OnTime_Weather_DFW_DEST_10_19.to_parquet(basepath + "/OnTime/OnTime_Weather_DFW_DEST_10_19_bf.parquet")

dest_2023_dfs.drop(columns=DtTm_cols, inplace=True)
dest_2023_dfs.to_parquet(basepath + "/OnTime/OnTime_Weather_DFW_DEST_23_bf.parquet")
OnTime_Weather_DFW_DEST_All_Yrs = pd.concat([OnTime_Weather_DFW_DEST_10_19, dest_2023_dfs], axis=0)
OnTime_Weather_DFW_DEST_All_Yrs.to_parquet(basepath + "/OnTime/OnTime_Weather_DFW_DEST_All_Yrs_bf.parquet")

# Final Dup check...

In [ ]:
OnTime_Weather_DFW_DEST_All_Yrs.columns

Index(['Dest', 'FlightDate', 'CRSDepTime', 'OriginDtTm', 'PredictTime',
       'w_dir_angle', 'w_type', 'w_speed_rate', 'sky_c_hgt', 'sky_c_det',
       'sky_c_cavok', 'vis_dist', 'vis_var', 'tmp_air', 'tmp_dew', 'sea_lvl_p',
       'liq_precip_qty', 'liq_precip_dim', 'liq_precip_cond', 'sky_cov',
       'sky_cov_base_hgt', 'sky_cov_cld', 'sky_sum_cov', 'sky_sum_hgt',
       'sky_obs_tot_cov', 'sky_low_cld_base_hgt', 'at_pres_altimeter_rate',
       'at_pres_stn_rate'],
      dtype='object')

In [ ]:
print(OnTime_Weather_DFW_DEST_All_Yrs.shape)
dupcheck = OnTime_Weather_DFW_DEST_All_Yrs[OnTime_Weather_DFW_DEST_All_Yrs.duplicated(subset=['Dest', 'OriginDtTm'], keep=False)]
print(dupcheck.shape)
dupcheck = OnTime_Weather_DFW_ORIG_All_Yrs[OnTime_Weather_DFW_ORIG_All_Yrs.duplicated(subset=['Origin', 'OriginDtTm'], keep=False)]
print(dupcheck.shape)

(1035955, 28)
(0, 28)
(0, 28)


# and check for missing values..

In [ ]:
print("OnTime_Weather_DFW_ORIG_10_19", OnTime_Weather_DFW_ORIG_10_19.columns[OnTime_Weather_DFW_ORIG_10_19.isnull().any()])
print("orig_2023_dfs", orig_2023_dfs.columns[orig_2023_dfs.isnull().any()])
print("OnTime_Weather_DFW_ORIG_All_Yrs", OnTime_Weather_DFW_ORIG_All_Yrs.columns[OnTime_Weather_DFW_ORIG_All_Yrs.isnull().any()])

print("OnTime_Weather_DFW_DEST_10_19", OnTime_Weather_DFW_DEST_10_19.columns[OnTime_Weather_DFW_DEST_10_19.isnull().any()])
print("dest_2023_dfs", dest_2023_dfs.columns[dest_2023_dfs.isnull().any()])
print("OnTime_Weather_DFW_DEST_All_Yrs", OnTime_Weather_DFW_DEST_All_Yrs.columns[OnTime_Weather_DFW_DEST_All_Yrs.isnull().any()])

OnTime_Weather_DFW_ORIG_10_19 Index([], dtype='object')
orig_2023_dfs Index([], dtype='object')
OnTime_Weather_DFW_ORIG_All_Yrs Index([], dtype='object')
OnTime_Weather_DFW_DEST_10_19 Index([], dtype='object')
dest_2023_dfs Index([], dtype='object')
OnTime_Weather_DFW_DEST_All_Yrs Index([], dtype='object')
